In [ ]:
import time
import random
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
my_ygo_expansions = ['2018-MegaTin-Mega-Pack',
 '2019-Gold-Sarcophagus-Tin-Mega-Pack',
 '2020-Tin-of-Lost-Memories-Mega-Pack',
 '2021-Tin-of-Ancient-Battles',
 '2022-Ghosts-From-the-Past',
 '2022-Tin-of-the-Pharaohs-Gods',
 '20th-Anniversary-Legend-Collection',
 '25th-Anniversary-Rarity-Collection',
 '25th-Anniversary-Tin-Dueling-Heroes-Mega-Pack',
 '2Player-Starter-Deck-Yuya-Declan',
 '3D-Bonds-Beyond-Time-Movie-Pack',
 'Age-of-Overlord',
 'Amazing-Defenders',
 'Ancient-Prophecy',
 'Battle-Pack-2-War-of-the-Giants',
 'Battle-Pack-Epic-Dawn',
 'Battle-of-Chaos',
 'Battles-of-Legend-Armageddon',
 'Battles-of-Legend-Crystal-Revenge',
 'Battles-of-Legend-Heros-Revenge',
 'Battles-of-Legend-Lights-Revenge',
 'Battles-of-Legend-Monstrous-Revenge',
 'Battles-of-Legend-Relentless-Revenge',
 'Blazing-Vortex',
 'Breakers-of-Shadow',
 'Brothers-of-Legend',
 'Burst-of-Destiny',
 'Circuit-Break',
 'Code-of-the-Duelist',
 'Collectors-Tins-2011',
 'Cybernetic-Horizon',
 'Cyberstorm-Access',
 'Dark-Crisis-25th-Anniversary-Edition',
 'Dark-Saviors',
 'Darkwing-Blast',
 'Dawn-of-Majesty',
 'Demo-Pack',
 'Dimension-Force',
 'Duel-Devastator',
 'Duel-Overload',
 'Duel-Power',
 'Duelist-Entry-Deck-VS',
 'Duelist-Nexus',
 'Duelist-Pack-Jaden-Yuki-2',
 'Duelist-Pack-Jesse-Anderson',
 'Duelist-Pack-Kaiba',
 'Duelist-Pack-Zane-Truesdale',
 'Egyptian-God-Deck-Obelisk-the-Tormentor',
 'Egyptian-God-Deck-Slifer-the-Sky-Dragon',
 'Eternity-Code',
 'Extreme-Force',
 'Fists-of-the-Gadgets',
 'Flames-of-Destruction',
 'Force-of-the-Breaker',
 'Fusion-Enforcers',
 'Genesis-Impact',
 'Hidden-Arsenal-Chapter-1',
 'Hidden-Summoners',
 'Ignition-Assault',
 'Invasion-of-Chaos-25th-Anniversary-Edition',
 'Kings-Court',
 'Legacy-of-Destruction',
 'Legend-of-Blue-Eyes-White-Dragon-25th-Anniversary-Edition',
 'Legendary-Collection-2-Mega-Pack',
 'Legendary-Collection-2-The-Duel-Academy-Years',
 'Legendary-Collection-3-Mega-Pack',
 'Legendary-Collection-Kaiba-Mega-Pack',
 'Legendary-Decks-II',
 'Legendary-Dragon-Decks',
 'Legendary-Duelists-9',
 'Legendary-Duelists-Ancient-Millennium',
 'Legendary-Duelists-Immortal-Destiny',
 'Legendary-Duelists-Magical-Hero',
 'Legendary-Duelists-Rage-of-Ra',
 'Legendary-Duelists-Season-1',
 'Legendary-Duelists-Season-2',
 'Legendary-Duelists-Season-3',
 'Legendary-Duelists-Sisters-of-the-Rose',
 'Legendary-Duelists-Synchro-Storm',
 'Legendary-Hero-Decks',
 'Light-of-Destruction',
 'Lightning-Overdrive',
 'Lost-Art-Promos',
 'Magicians-Force',
 'Magnificent-Mavens',
 'Maximum-Crisis',
 'Maximum-Gold',
 'Maximum-Gold-El-Dorado',
 'Metal-Raiders-25th-Anniversary-Edition',
 'Millennium-Pack',
 'Mystic-Fighters',
 'OTS-Tournament-Pack-10',
 'OTS-Tournament-Pack-13',
 'OTS-Tournament-Pack-15',
 'OTS-Tournament-Pack-16',
 'OTS-Tournament-Pack-18',
 'OTS-Tournament-Pack-19',
 'OTS-Tournament-Pack-20',
 'OTS-Tournament-Pack-7',
 'Phantom-Darkness',
 'Phantom-Nightmare',
 'Phantom-Rage',
 'Pharaohs-Servant-25th-Anniversary-Edition',
 'Photon-Hypernova',
 'Power-of-the-Duelist',
 'Power-of-the-Elements',
 'Premium-Gold',
 'Premium-Gold-Infinite-Gold',
 'Premium-Pack-2',
 'Ra-Yellow-Mega-Pack',
 'Return-of-the-Duelist',
 'Rise-of-the-Duelist',
 'Rising-Rampage',
 'Savage-Strike',
 'Scars-of-Battle',
 'Secret-Slayers',
 'Soul-Fusion',
 'Speed-Duel-Battle-City-Box',
 'Speed-Duel-GX-Duel-Academy-Box',
 'Spell-Ruler-25th-Anniversary-Edition',
 'Star-Pack-2013',
 'Star-Pack-VRAINS',
 'Starter-Deck-Codebreaker',
 'Starter-Deck-Jaden-Yuki',
 'Starter-Deck-Kaiba-Evolution',
 'Starter-Deck-Kaiba-Reloaded',
 'Starter-Deck-Link-Strike',
 'Starter-Deck-Xyz-Symphony',
 'Structure-Deck-Albaz-Strike',
 'Structure-Deck-Beware-of-Traptrix',
 'Structure-Deck-Cyber-Dragon-Revolution',
 'Structure-Deck-Cyber-Strike',
 'Structure-Deck-Cyberse-Link',
 'Structure-Deck-Dinosaurs-Rage',
 'Structure-Deck-Dinosmashers-Fury',
 'Structure-Deck-Emperor-of-Darkness',
 'Structure-Deck-Fury-From-The-Deep',
 'Structure-Deck-HERO-Strike',
 'Structure-Deck-Legend-of-the-Crystal-Beasts',
 'Structure-Deck-Lost-Sanctuary',
 'Structure-Deck-Master-of-Pendulum',
 'Structure-Deck-Onslaught-of-the-Fire-Kings',
 'Structure-Deck-Powercode-Link',
 'Structure-Deck-Rise-of-the-Dragon-Lords',
 'Structure-Deck-Sacred-Beasts',
 'Structure-Deck-Seto-Kaiba',
 'Structure-Deck-Shaddoll-Showdown',
 'Structure-Deck-Soulburner',
 'Structure-Deck-Spellcasters-Command',
 'Structure-Deck-Spirit-Charmers',
 'Structure-Deck-Synchron-Extreme',
 'Structure-Deck-The-Dark-Emperor',
 'Structure-Deck-The-Realm-of-Light',
 'Structure-Deck-Wave-of-Light',
 'Structure-Deck-Zombie-Horde',
 'Structure-Deck-Zombie-World',
 'Super-Starter-SpaceTime-Showdown',
 'Tactical-Masters',
 'The-Dark-Side-of-Dimensions-Movie-Pack',
 'The-Grand-Creators',
 'The-Secret-Forces',
 'Toon-Chaos',
 'Wild-Survivors',
 'Wing-Raiders']
expansions = []
cards = []
category = "Singles"
tcg = 'YuGiOh'
refresh_price = True

In [ ]:
counter_threshold = 20

In [ ]:
cardlist_file_path = f"{os.getcwd()}/data/{tcg}/cardlist.json"
updated_cards_prices_file_path = f"{os.getcwd()}/data/{tcg}/updated-cards-prices.json"
cards_to_keep_file_path = f"{os.getcwd()}/data/{tcg}/cards-to-keep.json"
export_file_path = f"{os.getcwd()}/data/{tcg}/collection_worth.xlsx"

In [ ]:
URL = f"https://www.cardmarket.com/en/{tcg}/Products"
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}

In [ ]:
def load_data():
    try:
        with open(cardlist_file_path, "r") as file:
            data = json.load(file)
    except Exception as e:
        print(f"Error loading {cardlist_file_path}. error: {e}")
        print(f"Data is an empty dict")
        data = {}
    return data

In [ ]:
def save_data(data):
    sorted_data = {
        category: {
            exp_name: {
                card_name: card_data
                for card_name, card_data in sorted(exp_data.items())
            }
            for exp_name, exp_data in sorted(category_data.items())
        }
        for category, category_data in sorted(data.items())
    }
    with open(cardlist_file_path, "w") as file:
        json.dump(sorted_data, file, indent=4)

In [ ]:
data = load_data()

In [ ]:
def setup_driver():
    options = Options()
    # options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.set_window_size(random.randint(800, 1200), random.randint(600, 800))
    actions = ActionChains(driver)
    print("Driver set")
    
    # Accepting Cookies
    driver.get(URL)
    cookie_button = """//*[@id="CookiesConsent"]/div/div/form/div/button"""
    cookie_btn = driver.find_element(By.XPATH, cookie_button)
    random_delay(4, 6)
    try:
        cookie_btn.click()
        print("Cookie Accepted")
        random_delay(2, 3)
    except Exception as e:
        print(f'Could not click on cookie button {URL}: {e}')
    return driver, actions

In [ ]:
def random_delay(min_delay=3, max_delay=10):
    time.sleep(random.uniform(min_delay, max_delay))

In [ ]:
try:
    with open(updated_cards_prices_file_path, "r") as file:
        updated_cards_prices = json.load(file)
except Exception as e:
    updated_cards_prices = {}

In [ ]:
def save_updated_cards_prices(updated_cards_prices):
    os.makedirs(os.path.dirname(updated_cards_prices_file_path), exist_ok=True)
    with open(updated_cards_prices_file_path, "w") as file:
        json.dump(updated_cards_prices, file, indent=4)

In [ ]:
price_from = """//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[6]"""
price_trend = """//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[7]/span"""
price_30_day_avg = """//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[8]/span"""

In [ ]:
def convert_currency_string(currency_str):
    if '£' in currency_str and '.' in currency_str and ',' in currency_str:
        currency_str = str(float(currency_str.strip('£').replace('.', '').replace(',', '.'))*1.16).replace('.', ',')
    elif '£' in currency_str:
        currency_str = str(float(currency_str.strip('£').replace(',', '.'))*1).replace('.', ',')
    cleaned_str = currency_str.strip('€ £')
    
    # If there's a comma and no period, it's likely a decimal comma
    if ',' in cleaned_str and '.' not in cleaned_str:
        cleaned_str = cleaned_str.replace(',', '.')
    else:
        # Remove periods used as thousand separators
        cleaned_str = cleaned_str.replace('.', '').replace(',', '.')

    return float(cleaned_str)

In [ ]:
def get_df(data, category, selected_expansions = [], selected_cards = []):
    cards = []
    for expansion, card_list in data[category].items():
        for card_name, card_info in card_list.items():
            card_info_copy = card_info.copy()  # Avoid modifying the original data
            card_info_copy.update({'expansion': expansion, 'card_name': card_name})
            cards.append(card_info_copy)

    df = pd.DataFrame(cards).set_index(['expansion', 'card_name'])

    if selected_expansions or selected_cards:
        df = df.loc[df.index.get_level_values('expansion').isin(selected_expansions) | 
                    df.index.get_level_values('card_name').isin(selected_cards)].copy()
    
    df = df[df['quantity'] > 0]
    
    return df

In [ ]:
def get_price(data, category, selected_expansions = [], selected_cards = []):
    global driver, actions
    df = get_df(data, category, selected_expansions, selected_cards)

    counter = 0
    for index, row in df.iterrows():
        if counter == counter_threshold:
            save_data(data)
            save_updated_cards_prices(updated_cards_prices)
            driver.quit()
            driver, actions = setup_driver()
            counter = 0
        expansion, card_name = index
        if expansion in updated_cards_prices and card_name in updated_cards_prices[expansion]:
            continue
        print(f"Getting price for: {card_name}")
        url = f"{URL}/{category}/{expansion}/{card_name}"
        driver.get(url)
        random_delay(1, 3)
        # actions.move_by_offset(random.randint(0, 100), random.randint(0, 100)).perform()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        random_delay(3, 4)
        
        try:
            p_from = convert_currency_string(driver.find_element(By.XPATH, price_from).text)
            df.at[index, 'price_from'] = p_from
            data[category][expansion][card_name]['price_from'] = p_from
        except Exception as e:
            print(f'Could not retrieve price_from for {url}: {e}')
        try:
            p_trend = convert_currency_string(driver.find_element(By.XPATH, price_trend).text)
            data[category][expansion][card_name]['price_trend'] = p_trend
        except Exception as e:
            print(f'Could not retrieve price_trend for {url}: {e}')
        try:
            p_30_day = convert_currency_string(driver.find_element(By.XPATH, price_30_day_avg).text)
            df.at[index, 'price_30_day_avg'] = p_30_day
            data[category][expansion][card_name]['price_30_day_avg'] = p_30_day
        except Exception as e:
            print(f'Could not retrieve price_30_day_avg for {url}: {e}')

        if expansion in updated_cards_prices:
            updated_cards_prices[expansion].append(card_name)
        else:
            updated_cards_prices[expansion] = [card_name]
        counter += 1
    save_data(data)
    save_updated_cards_prices(updated_cards_prices)

In [ ]:
# print(convert_currency_string('£1.999,99'))
# print(convert_currency_string('£0,17'))
# print(convert_currency_string('£0.02'))
# print(convert_currency_string('0,317 €'))
# print(convert_currency_string('1.999,99  €'))

In [ ]:
driver = None
if refresh_price:
    driver, actions = setup_driver()
    get_price(data, category, expansions, cards)

In [ ]:
pricing_basis = "price_trend"

In [ ]:
def get_worth(df, pricing_basis=pricing_basis, export=False):
    df['total_price_from'] = df['quantity'] * df['price_from']    
    df['total_price_trend'] = df['quantity'] * df['price_trend']    
    df['total_price_price_30_day_avg'] = df['quantity'] * df['price_30_day_avg']    
    
    summary_s = df[['quantity', 'total_price_from', 'total_price_trend', 'total_price_price_30_day_avg']].sum()
    print(summary_s)
    
    summary_s['expansion'] = 'Total'
    summary_s['card_name'] = 'Total'
    summary = summary_s.to_dict()
    
    # Create a DataFrame for the totals row and concatenate it to the original DataFrame
    summary_df = pd.DataFrame([summary]).set_index(['expansion', 'card_name'])
    export_df = pd.concat([df, summary_df])
    
    # Export to Excel if specified
    if export:
        print(f"Exported to {export_file_path}")
        export_df.to_excel(export_file_path, float_format='%.2f')


In [ ]:
df = get_df(data, category, expansions, cards)

In [ ]:
get_worth(df, export=True)

In [ ]:
bulk_df = df[df['price_trend'] <= 0.5]
get_worth(bulk_df)

In [ ]:
not_bulk_df = df[df['price_trend'] > 0.5]
get_worth(not_bulk_df)

In [ ]:
try:
    with open(cards_to_keep_file_path, "r") as file:
        cards_to_keep = json.load(file)
except Exception as e:
    cards_to_keep = {}

In [ ]:
filter = ['Neos','Rainbow-Dragon']

In [ ]:
mask = df.index.get_level_values(1).str.contains('|'.join(filter), case=False, na=False)
filtered_df = df[mask]
get_worth(filtered_df)

In [ ]:
for c in list(filtered_df.index):
    if c[0] in cards_to_keep:
        if c[1] not in cards_to_keep[c[0]]:
            print(f"{c[0]}: {c[1]}")
            cards_to_keep[c[0]].append(c[1])
    else:
        cards_to_keep[c[0]] = [c[1]]

In [ ]:
def save_cards_to_keep(cards_to_keep):
    os.makedirs(os.path.dirname(cards_to_keep_file_path), exist_ok=True)
    with open(cards_to_keep_file_path, "w") as file:
        json.dump(cards_to_keep, file, indent=4)

In [ ]:
save_cards_to_keep(cards_to_keep)

In [ ]:
# df[(df['price_from'] > df['price_trend']*1.2) & (df['quantity'] > 0)]

In [ ]:
if driver:
    driver.quit()